# Include 

In [1]:
import json
import numpy as np
import os
import random

from os.path import expanduser

# Definitions 

# Main course

## Load geography

In [2]:
base = expanduser("~/database/shad/generate_text/ru")

In [3]:
filenames = ["city", "hamlet", "town", "village"]

In [4]:
dataset_raw = [json.loads(line) for current_name in filenames for line in open(os.path.join(base, f"place-{current_name}.ndjson"))]

In [5]:
len(dataset_raw)

253192

In [6]:
sum(1 for r in dataset_raw if "name" not in r)

57

In [7]:
names = [r["name"] for r in dataset_raw if "name" in r]

In [8]:
len(names)

253135

In [9]:
uniq_names = list(set(names))

In [10]:
len(uniq_names)

77411

In [11]:
random.shuffle(uniq_names)

In [12]:
uniq_names[:10]

['Паровичи',
 'Первуниха',
 'Аношкино',
 'Буцра',
 'Нижний Калгукан',
 'Качка',
 'Гольцево',
 'Знамя Октября',
 'Нижняя Гора',
 'Шарлово']

In [13]:
alphabet=sorted(list(set(c for r in uniq_names for c in r)))

In [14]:
"".join(alphabet)

' "(),-./0123456789IVXc\xa0«»́ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№'

In [15]:
ss_alphabet = alphabet + ["<", ">"]

In [46]:
c2ind = {c:ind for ind, c in enumerate(ss_alphabet)}
ind2c = {ind:c for ind, c in enumerate(ss_alphabet)}

In [17]:
ss_names = ["<" + v + ">" for v in uniq_names]

In [18]:
ss_names[:10]

['<Паровичи>',
 '<Первуниха>',
 '<Аношкино>',
 '<Буцра>',
 '<Нижний Калгукан>',
 '<Качка>',
 '<Гольцево>',
 '<Знамя Октября>',
 '<Нижняя Гора>',
 '<Шарлово>']

# Create dataset

In [19]:
text = "".join(ss_names)
len(text)

892218

In [20]:
text[:4000]

'<Паровичи><Первуниха><Аношкино><Буцра><Нижний Калгукан><Качка><Гольцево><Знамя Октября><Нижняя Гора><Шарлово><Лукново><Деревянное><Петуховское><Жаруха><Столбовка><Табатай><Исади><Петрушиха><Лычное><Кадряково><Малый Бащелак><Митянино><Щигры><Исаково><Мамакино><Орсино><Мордвиново><Алешкин><Большие Слудицы><Зелёный Городок><Гороховщино><Мордовинка><Мустафино><Старошешминск><Голышино><Велема><Гарах><Волыны><Бейсугское><Моховые Липки><Медвежьегорск><Иваново-Наумовка><Жиндо 2-е><Ашлань-Вершина><Венедеевка><Бабурино><Копытово><Малое Погорелово><Пестово><Филатова Гора><Пичер><Большекрутинская><Калачиха><Ближняя Кубасова><Карабаши><Макарьевка><Шитхулун><Новогородец><Карды><Андронята><Разветье><Старинка><Кошки><Семейкино><Верхнее Вельяминово><Моченки><Соголево><Поломеницы><Девкин><Подоклинье><Скалябино><Захаровский Выставок><Гнездино><Подорки><Цагакшино><кутан совхоза имени 8 Марта><Толтуново><Селютинская><Желомеено><Шалана><Балыково><Грушевое><Тимофеево><Поспелова><Русская Селитьба><Новое Эште

# Forward pass

## Create a model

In [117]:
hsz = 100 # hidden size
vsz = len(ss_alphabet) # vocabulary size
seq_length = 25 # the size of window
lr = 0.1

In [118]:
Wxh = np.random.randn(hsz, vsz) * 0.01
Whh = np.random.randn(hsz, hsz) * 0.01
Why = np.random.randn(vsz, hsz) * 0.01
bh = np.zeros((hsz, 1))
by = np.zeros((vsz, 1))

In [119]:
def gen_portion(text, seq_length):
    n = len(text)    
    epoch = 0
    while True:        
        start = epoch % seq_length
        while start + seq_length <= n:
            yield epoch, text[start: start + seq_length]
            start += seq_length
        epoch += 1

In [120]:
for epoch, portion in gen_portion("abcd",3):
    print(epoch, portion)
    if epoch > 5:
        break

0 abc
1 bcd
3 abc
4 bcd
6 abc


In [121]:
def sample(h, t=1):
  x = np.zeros((vsz, 1))
  x[c2ind['<']] = 1
  s = []
  while True:
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y/t) / np.sum(np.exp(y/t))
    ix = np.random.choice(range(vsz), p=p.ravel())
    x = np.zeros((vsz, 1))
    x[ix] = 1
    next_char = ind2c[int(ix)]
    if next_char != ">":
        s.append(next_char)
    else:
        return "".join(s)

In [122]:
def calcLoss(ind_input, ind_target, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    for t in range(len(ind_input)):
        xs[t] = np.zeros((vsz, 1))
        xs[t][ind_input[t]] = 1
        hs[t] = np.tanh(Wxh@xs[t] + Whh@hs[t-1] + bh)
        ys[t] = Why@hs[t] + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss -= np.log(ps[t][ind_target[t], 0])

    dWhh, dWxh, dWhy = np.zeros_like(Whh), np.zeros_like(Wxh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(ind_input))):
        dy = np.copy(ps[t])
        dy[ind_target[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += dy@hs[t].T
        dby += dy
        dh = Why.T @ dy + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += dhraw @ xs[t].T
        dWhh += dhraw @ hs[t-1].T
        dhnext = Whh.T @ dhraw
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[-1]

In [123]:
hprev = np.zeros((hsz, 1))
smooth_loss = None
print("epoch, loss, sample")

mWhh, mWxh, mWhy, mbh, mby = (np.zeros_like(a) for a in [Whh, Wxh, Why, bh, by])

for ind, (epoch, portion) in enumerate(gen_portion(text, seq_length)):
    text_input, text_target = portion[:-1], portion[1:]
    ind_input = [c2ind[c] for c in text_input]
    ind_target = [c2ind[c] for c in text_target]
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = calcLoss(ind_input, ind_target, hprev)
    smooth_loss = loss if smooth_loss is None else 0.999 * smooth_loss + 0.001 * loss
    if ind % 2000 == 0:
        print(epoch, loss, sample(hprev))

    # for param, dparam in [[Wxh, dWxh], [Whh, dWhh], [Why, dWhy], [bh, dbh], [by, dby]]:
    #    param -= lr * dparam

    for param, dparam, mem in [[Wxh, dWxh, mWxh], [Whh, dWhh, mWhh], [Why, dWhy, mWhy], [bh, dbh, mbh], [by, dby, mby]]:
        mem += dparam * dparam
        param -= lr * dparam / np.sqrt(mem + 1e-8)

    if epoch > 10:
        break

epoch, loss, sample
0 108.78798678523576 1Ч,ДоДп"дхНщVьдВ9VАВьжцгЕ Ы(х8ЦИ лкЩнде,Оц0еЗЁ5ШААЯ8пнюПЯV́итзкэаОЦДтМ́ П́н3дгЭвФоХлС/ФVмжыВМ́/Xгщ(Мюи1"-c)ЯаИЯЖТIИ пБХ(«2пшЕйХОлФУАЛИъ-/,ЦыаППcоЧ, РЧфЗп1ёсйЫ2Т8К-9фо-аилПюБргРкъ»чэМж"8ЛяПЮ Iй ЁеаХЗз06ф9ИиСр)З«2рХ аЖК6оАcКХБхОV.ШчэеЫ»ъу.иц№охаXЛф
0 45.47149012560989 Нодоо
0 40.55481954254759 Зололос
0 47.85341904415365 Митёний
0 46.623290496542594 Старито
0 54.40061842601707 Канинов
0 48.352443387343946 Сучва
0 38.86232278042936 Руга
0 45.59988742228374 Дяминог
0 43.897572267518875 Труй
0 52.569552081773125 Сеихи
0 36.531769187825525 Кумгмгух
0 40.32554400118668 Крисылново
0 49.78545274984682 Тееданая Мыги
0 43.46473539311892 Зийра
0 51.58721905767358 Бортово
0 44.00332388070326 Зполиные Юврены
0 39.749417506125084 Верхнее
1 31.120755924801866 Эльва
1 64.0058806090256 Дшепзумаево
1 51.63772704402982 Лилачёвское
1 40.99213006602315 Зепрудрах
1 38.766243273867595 Кутюлка
1 53.16553790364616 Балеезёво
1 48.49419368387541 Гонченое
1 35.7909250966686

KeyboardInterrupt: 

In [112]:
smooth_loss = None
print("epoch, loss, sample")
for ind, (epoch, portion) in enumerate(gen_portion(text, seq_length)):
    text_input, text_target = portion[:-1], portion[1:]
    ind_input = [c2ind[c] for c in text_input]
    ind_target = [c2ind[c] for c in text_target]
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = calcLoss(ind_input, ind_target, hprev)
    smooth_loss = loss if smooth_loss is None else 0.999 * smooth_loss + 0.001 * loss
    if ind % 2000 == 0:
        print(epoch, loss, sample(hprev))

    for param, dparam in [[Wxh, dWxh], [Whh, dWhh], [Why, dWhy], [bh, dbh], [by, dby]]:
        param -= lr * dparam

    if epoch > 10:
        break

epoch, loss, sample
0 55.37196802098065 0ертин
0 49.53649614546994 тна
0 44.53878607394509 Ктыелнки
0 52.68488442382352 ава
0 56.50003217400247 Норниф<масидино
0 62.64589050651468 Ткрковово
0 58.58059488249559 рёминь1
0 51.20251974027695 туна
0 49.316647800019666 Бхаровы
0 60.527081044095276 мулвсолака
0 62.64475760306713 ТулопхГнея Ввоштово
0 42.61569014020909 юмкоцехови
0 49.72708715268235 яйнсбимово
0 56.76925572987482 итудичи
0 49.39672666152903 гсайевдашово
0 56.51294318985003 Биним
0 59.274212094067536 Евамово
0 42.148187640861394 овакий -рорка
1 44.367506729489286 Гсоруготоево
1 60.787506265043376 Ераловона
1 55.99066886462775 Каньт
1 45.41392537262714 саиланетсы
1 59.39427129033753 галулорово
1 57.57798095672117 Влино вол темечка
1 60.27769144939958 харьюСавый Дево
1 47.992405575491624 певтеча
1 61.07772249904601 Ёе4дурсяво
1 49.49687753087553 етелакамово
1 55.42905840578496 дорыкое Шм-шинтЛ-2а
1 55.93135831096905 3огсшнков
1 57.451603196071346 Еомовсяака
1 51.81874647949457 Бу

In [113]:
for _ in range(10):
    print(sample(hprev, 1))

Сендичихо
лоноебов
атповка
Ганое Корцаровк
Мацёкажсонския Пмсаябевка
Iьчянкод
СратоедСтово
чиньеруно
Еевь-Ленное
Бучое Пуленое Немевьа


In [124]:
for _ in range(10):
    print(sample(hprev, 1))

Большое Алкова
Стряновка
Мухантино
Бугоо
Ишково
Угиньевка
Ряшсино
Стровский
Милицы
Бжубиги


In [114]:
for _ in range(10):
    print(sample(hprev, 0.1))

Кареново
Караново
Караново
Карерово
Сараново
Карово
Караново
Караново
Борово
Караново


In [125]:
for _ in range(10):
    print(sample(hprev, 0.1))

Старое Колово
Караманово
Старое Колово
Караново
Малое Старое Колово
Старое Колово
Караки
Старое Колово
Караманово
Старое Колово


In [126]:
for _ in range(10):
    print(sample(hprev, 0.05))

Каранино
Караманово
Караново
Старое Колово
Старое Колово
Караново
Старое Караново
Караново
Караманово
Старое Колово


In [127]:
for _ in range(30):
    print(sample(hprev, 0.6))

Большая Арсконово
Остринский
Старый Винова
Маный Субино
Поторовщина
Нижняя Имайково
Мархатар
Коново
Красный Ардар
Новолодеково
Шеманово
Растаставкино
Новокорки
Булаш
Новое Алановский
Короговая
Полилино
Ладаки
Колобово
Верхний Озеровка
Кочанка
Сарабовка
Минченки
Горово
Шерваши
Красной Сухово
Малая Яновино
Горатаново
Мелогерки
Новокосель


In [128]:
for _ in range(30):
    print(sample(hprev, 1.2))

Рабожец
Обаста
Затиевацкиша
Ермы
(Лёшковский
Маны-Росоводое
Брудачки
Пулгищево Вылёво
ВЧизоров Замырево
Никрабное
ВиПори
Испошеловск
Шерное
Морокайво
Либа
Вае-Жугозок
Клевня
Бтыксиха
Капловка
Нахлимригары
Беихщи
Ледюница
Балтелядловный
Зетбобамово
Кулганабовлкий
Тимай
Исок
Щапрово
паголол
ПроговнифРис


In [129]:
for _ in range(30):
    print(sample(hprev, 1.4))

Увичи
Хорёихуни
Заюшевское-плухово
Шазюково
Турвилово
Страгешодм
Исве1
МихануБргап
Лак-Шайсьево
Варвочнево
Микаозовка
Чеешлево
Красная Шайневское
Малаевский
Зебкша
Алсахный
Кысымс
Жиган-Еньва
Уйзовоа
Илыма Лоския
Конкино
22 кмищено
Баяльловка
Ороновка
Болбыпина
Юбожское
Булицкая
Марыголилово-ЭМагезе
морошков
Калниг
